In [ ]:
# This Python 3 environment comes with many helpful analytics libraries installed
# It is defined by the kaggle/python Docker image: https://github.com/kaggle/docker-python
# For example, here's several helpful packages to load

import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)

# Input data files are available in the read-only "../input/" directory
# For example, running this (by clicking run or pressing Shift+Enter) will list all files under the input directory

import os
for dirname, _, filenames in os.walk('/kaggle/input'):
    for filename in filenames:
        print(os.path.join(dirname, filename))

# You can write up to 20GB to the current directory (/kaggle/working/) that gets preserved as output when you create a version using "Save & Run All" 
# You can also write temporary files to /kaggle/temp/, but they won't be saved outside of the current session

# Importing Necessary Libraries

In [ ]:
import numpy as np
import pandas as pd 
import matplotlib.pyplot as plt
import plotly.express as px
import seaborn as sns
%matplotlib inline
from sklearn.preprocessing import LabelEncoder
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LinearRegression
from sklearn.ensemble import RandomForestRegressor
import sklearn.metrics as metrics
import os

# Reading the Dataset

In [ ]:
data = pd.read_csv('/kaggle/input/pizza-price-prediction/pizza_v2.csv')

In [ ]:
data.head()

# Analyzing Preparing the Dataset

In [ ]:
data.isnull().sum()

### No Missing Values

In [ ]:
data.describe

In [ ]:
data.info()

### All object values

### Convering Object data like Price and Diameter to numerical 

In [ ]:
data.head()

In [ ]:
data['price_rupiah'] = data['price_rupiah'].apply(lambda x : x.replace("Rp","").replace(",","."))
# remove rp and comma and store it in new price column
data['diameter'] = data['diameter'].apply(lambda x : x.replace(" inch",""))
#remove inch and store it in new column

In [ ]:
data.head()

In [ ]:
data["diameter"] = data["diameter"].astype(float)
data["price_rupiah"] = data["price_rupiah"].astype(float)

In [ ]:
data.info()

In [ ]:
data.head()

### Checking for categories under each categorical feature


In [ ]:
cat_col = [col for col in data.columns if data[col].dtype == 'object']
num_col = [col for col in data.columns if data[col].dtype != 'object']

In [ ]:
for col in cat_col:
    print(f'A {col} has {data[col].unique()} list of values and the number is {data[col].nunique()}\n unique values \n')

### Encoding categorical features

In [ ]:
data.info()

In [ ]:
#OneHot won't be a good idea coz there are too less rows
en = LabelEncoder()
#catCols = ['company','topping','variant','size','extra_sauce','extra_cheese','extra_mushrooms']
for cols in cat_col:
    data[cols] = en.fit_transform(data[cols])

In [ ]:
data.head()

In [ ]:
data.info()

### Whole dataset is in numerical form now!!
### 
### 


# Splitting into X and Y

In [ ]:
Y = data["price_rupiah"]
X = data.drop("price_rupiah",axis = 1)

In [ ]:
X.head()

In [ ]:
Y.head()

In [ ]:
Y=pd.DataFrame(Y)

In [ ]:
Y.head()

# Test-Train Split

In [ ]:
# Break off validation set from training data
X_train_full, X_valid_full, Y_train, Y_valid = train_test_split(X, Y, train_size=0.8, test_size=0.2,
                                                                random_state=0)

In [ ]:
X_train_full.head()

In [ ]:
Y_train.head()

In [ ]:
sns.pairplot(data)

Only graph which seems interesting is price vs diameter which is quite what we expect naturally.

# XGB Regressor

In [ ]:
from xgboost import XGBRegressor
from sklearn.metrics import mean_absolute_error
# Define the model
model = XGBRegressor(n_estimators=1000, learning_rate=0.09) 

# Fit the model
model.fit(X_train_full, Y_train) 

# Get predictions
predictions = model.predict(X_valid_full) 

# Calculate MAE
mae = mean_absolute_error(predictions, Y_valid)
print("Mean Absolute Error:" , mae)

#Calculating R2
r2 =  metrics.r2_score(Y_valid, predictions)
print("R2 score :", r2)

In [ ]:
predictions = pd.DataFrame(predictions, columns=Y_valid.columns)
predictions

In [ ]:
Y_valid

In [ ]:
Y_valid.reset_index(drop=True, inplace=True)
final = pd.concat([predictions,Y_valid],axis=1)
final.columns = ['Predictions', 'Y_valid']
final

# Improved XGB egressor

In [ ]:
# Define the model
model2 = XGBRegressor(n_estimators=1000, learning_rate=0.01,max_depth=10) 

# Fit the model
model2.fit(X_train_full, Y_train) 

# Get predictions
predictions2 = model2.predict(X_valid_full) 

# Calculate MAE
mae2 = mean_absolute_error(predictions2, Y_valid)
print("Mean Absolute Error:" , mae2)

#Calculating R2
r22 =  metrics.r2_score(Y_valid, predictions2)
print("R2 score :", r22)

In [ ]:
Y_train.shape

In [ ]:
from sklearn.ensemble import RandomForestRegressor

rf = RandomForestRegressor(n_estimators=1000)

rf.fit(X_train_full, np.array(Y_train).ravel())

In [ ]:
rf.score(X_valid_full, Y_valid)

In [ ]:
metrics.r2_score(rf.predict(X_valid_full), Y_valid)

Seems that XGBReg is doing far better.